In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import time
import os
import numpy as np
import cv2
import random
# import pandas as pd

import json
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

def plot(src, cmap=None, title=None, size=(10, 10)):
    plt.rcParams['figure.figsize'] = size
    plt.imshow(src, cmap)
    plt.title(title)

In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')

test = pd.read_csv('../input/shopee-product-matching/test.csv')

train.head()

In [ ]:
groups = train.groupby('label_group')['posting_id'].apply(lambda x: ' '.join(x)).reset_index()
groups.columns = ['label_group', 'posting_id_list']
train = train.merge(groups)

In [ ]:
# train['label_group'].value_counts()

In [ ]:
# groups = train['label_group'].unique()
# big_groups = []
# for i in tqdm(groups):
#     group = train[train['label_group']== i]
#     if group.shape[0] > 10:
#         big_groups.append(group)
#     print(group['title'], group.shape)
    

In [ ]:
train_imgs_path = '../input/shopee-product-matching/train_images/'
test_imgs_path = '../input/shopee-product-matching/test_images/'

train_imgs = [i for i in os.listdir(train_imgs_path) if i.endswith('.jpg')]
test_imgs = [i for i in os.listdir(test_imgs_path) if i.endswith('.jpg')]
# print(len(test_imgs))

# img = cv2.imread(f"{train_imgs_path}{train_imgs[0]}")[:,:,::-1]
# plot(img)

In [ ]:
def show_imgs(imgs, labels, num_images: int = 4):
    """
    Функция для отображения батча картинок с леблами
    """
    f, axes= plt.subplots(num_images, 1, figsize=(200, 200))
    for i, axis in enumerate(axes):
        axes[i].imshow(imgs[i])
        if type(labels[i]) != str:
            axes[i].set_title(classes_reverse_encoder[int(labels[i])])
        else:
            axes[i].set_title(labels[i])
    plt.show()


In [ ]:
# group = big_groups[12]

# group_imgs = []
# labels = [title for title in group['title']]

# for img_name in group['image']:
#     img = cv2.imread(f'{train_imgs_path}{img_name}')[:,:,::-1]
#     group_imgs.append(img)
    
# show_imgs(group_imgs, labels=labels, num_images=len(group_imgs))

In [ ]:
# model = models.resnext101_32x8d(pretrained=False)
model = models.resnet18(pretrained=False)

In [ ]:
model

In [ ]:
model = nn.Sequential(*list(model.children())[:-1])

In [ ]:
# torch.save(model.state_dict(), 'resnet18.pt')

In [ ]:
model.load_state_dict(torch.load('../input/resnext/resnet18.pt'))
model.eval()


In [ ]:
model.to('cuda')

In [ ]:
transform = transforms.Compose([
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

In [ ]:
from PIL import Image
from tqdm.notebook import tqdm

In [ ]:
embeddings = []

for img_name in tqdm(test['image']):
    image = Image.open(f'{test_imgs_path}{img_name}')

    img = transform(image).unsqueeze(0).to('cuda')
    
    with torch.no_grad():
        out = model(img).squeeze().cpu().numpy()
        embeddings.append(out)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def get_matches(embedding):
    return ' '.join([posting_id[j] for j in np.argwhere(cosine_similarity(embedding, embeddings)[0] > 0.85).squeeze(1)]) 


In [ ]:
posting_id = test['posting_id'].tolist()

In [ ]:
# matches = [' '.join([posting_id[j] for j in np.argwhere(i > 0.85).squeeze(1)]) for i in similarities]
matches = []

for emb in tqdm(embeddings):
    match = get_matches(np.array([emb]))          
    matches.append(match)

In [ ]:
# f1_score = []
# for match, posting_id_list in zip(matches, train['posting_id_list'].tolist()):
#     match = set(match.split())
#     posting_id_list = set(posting_id_list.split())
#     precision = 1 - len(match - posting_id_list) / len(match)
#     recall = 1 - len(posting_id_list - match) / len(posting_id_list)
#     f1_score.append(2 * precision * recall / (precision + recall))
# f1_score = np.mean(f1_score)

In [ ]:
# f1_score

In [ ]:
submission = pd.DataFrame({'posting_id': posting_id, 'matches': matches})

In [ ]:
# submission

In [ ]:
submission.to_csv('submission.csv', index=False)